# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#Dependencies
import requests
import random
import numpy as np
import pandas as pd
from citipy import citipy
# Import API key
from api_keys import weather_api_key


## Generate Cities List

In [2]:
#lat south - 90 eq 0 north 90 55 139   21
#long -180 to 180
cities = []
while len(cities) < 10:
    lat = round(np.random.uniform(-90.0, 90.0),2)
    lon = round(np.random.uniform(-180.0, 180.0),2)
    city = citipy.nearest_city(lat, lon).city_name
    cities.append(city)
    cities = list(set(cities))
print(cities)
len(list(set(cities)))

['albany', 'busselton', 'barrow', 'bluff', 'bua yai', 'katete', 'ushuaia', 'avarua', 'new norfolk', 'sungaipenuh']


10

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:

print("Begining of data retrieval/n-----------------")

lat = []
lon = []
maxtemp = []
humidity = []
cloudiness =[]
wind = []
country = []
date = []
i = 1
for city in cities:

        
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units=metric&appid={weather_api_key}"
    #print(url)   
    data = requests.get(url).json()
    if data["cod"] == "404":
        message = data["message"]
        print(f"{message.capitalize()}. Skipping...")
    
    else:  
        print(f"Processing Record {i} of Set {len(cities)} | {city}")
        
        lat.append(data["coord"]["lat"])
        lon.append(data["coord"]["lat"])
        maxtemp.append(data["main"]["temp_max"])
        humidity.append(data["main"]["humidity"])
        cloudiness.append(data["clouds"]["all"])
        wind.append(data["wind"]["speed"])
        country.append(data["sys"]["country"])
        date.append(data["dt"])
        i +=1
citiesdata = {
    'City': cities,
    'Lat': lat, 
    'Lng': lon, 
    "Max Temp" : maxtemp, 
    "Humidity" : humidity, 
    "Cloudiness" : cloudiness, 
    "Wind Speed": wind, 
    "Country": country,
    "Date" : date
}
# "cod":200

# {"cod":"404","message":"city not found"}

Begining of data retrieval/n-----------------
Processing Record 1 of Set 10 | albany
Processing Record 2 of Set 10 | busselton
Processing Record 3 of Set 10 | barrow
Processing Record 4 of Set 10 | bluff
Processing Record 5 of Set 10 | bua yai
Processing Record 6 of Set 10 | katete
Processing Record 7 of Set 10 | ushuaia
Processing Record 8 of Set 10 | avarua
Processing Record 9 of Set 10 | new norfolk
Processing Record 10 of Set 10 | sungaipenuh


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
df1 = pd.DataFrame(citiesdata)
df1.head()

,lat,lon,maxtemp,humidity,cloudiness,wind,country,date
0,42.6001,42.6001,10.56,82,47,1.47,US,1618394581
1,-33.6500,-33.6500,16.67,69,15,5.34,AU,1618394515
2,71.2906,71.2906,-14.00,78,1,4.12,US,1618394334
3,-46.6000,-46.6000,10.56,93,99,1.68,NZ,1618394424
4,15.5855,15.5855,34.66,39,89,6.26,TH,1618394582


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.

    

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
def humidityoutliers():
    clean_city_data = df.

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression